In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
import time
import sys
import warnings
warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
from db.database import DatabaseConnector
from dtloader.dataloader import DataLoader
from failure_detector import *
from utils.utils import *

In [ ]:
dl = DataLoader()
dl.load('sample.log')
dl.extractinfo(export=False,single_file=True)

# Sample Log Analysis

In [ ]:
ts = {'RCIN':['C1','C2','C3','C4'] ,'AHR2':['Roll','Pitch','Alt','Yaw']}
df,df_corr = corr_var('sample.log',dl,ts)
df_corr.iloc[4:,:4]

In [ ]:
ts = {'MODE':['Mode'] ,'ERR':['Subsys']}
df,df_corr = corr_var('sample.log',dl,ts)
df_corr

# All Logs - Mean Correlation - RCIN : AHR2

In [ ]:
datalist =os.listdir('../data')

In [ ]:
ts = {'RCIN':['C1','C2','C3','C4'] ,'AHR2':['Roll','Pitch','Alt','Yaw']}
print('File:{} --  {} / {}'.format(datalist[0],0,len(datalist)-1))
d,df_cnc = corr_var(datalist[0][:-4],ts)
df_cnc = df_cnc.iloc[4:,:4]
count_list = []
for i,log in enumerate(datalist[1:]):
    print('File:{} --  {} / {}'.format(log,i+1,len(datalist)-1))
    count_novalues,df_corr = corr_var(log[:-4],dl,ts)
    df_cnc = pd.concat((df_cnc, df_corr.iloc[4:,:4]))
    count_list.append(count_novalues)
#compute the Mean 
by_row_index = df_cnc.groupby(df_cnc.index)
df_means = by_row_index.mean()
df_means

In [ ]:
df_means.dropna(axis=1,how='all')

In [ ]:
print('Mean Number of variables not found in log files : ',np.mean(count_list))

# Top 100 Variables Correlation Table

In [ ]:
df = load_top100()
#put the var in the right form for the method input
vardict = load_top100_dict()

In [ ]:
#Creating 100x100 correlation table
start_time = time.time()
df_cnc= pd.DataFrame({'temp':[0]}) #init table
for i,log in enumerate(datalist):
    print('File:{} --  {} / {}'.format(log,i,len(datalist)-1))
    _,df_corr = corr_var(log[:-4],dl,vardict)
    df_cnc = pd.concat((df_cnc, df_corr))
by_row_index = df_cnc.groupby(df_cnc.index)
df_means = by_row_index.mean()
df_means.drop([0],inplace=True)
df_means.drop('temp',axis=1,inplace=True)
print('Time Taken to load correlation Table :',time.time() - start_time)
print(df_means.shape)

In [ ]:
print(len(df_means))
np.setdiff1d(df['name'],df_means.columns)

In [ ]:
df_means.to_csv('corr_table.csv')

## Graphing Network

In [ ]:
dt = {}
threshold_corr = 0.6
for c in df_means.columns:
    idx = abs(df_means.loc[c,:]) >= threshold_corr
    df_means.columns[idx]
    dt[c] = df_means.columns[idx]

In [ ]:
def graph_generator(edges_dict):
        """
        A single graoh generator based on networkx
        :param nodes_list: The list of nodes. A simple list with strings for node names.
        :param edges_dict: A dict containing every node as key and a list containing other nodes as value. The key
        will be connected to the nodes in the value.
        """
        nodes_list = edges_dict.keys()
        plt.figure(figsize=(15,12)) 
        plt.clf()
        plt.axis("off")
        G = nx.Graph()
        for key in nodes_list:
            G.add_node(key)

        for key in edges_dict.keys():
            connect_to = edges_dict[key]
            for node in connect_to:
                G.add_edge(key, node)
        pos = nx.spring_layout(G)
        labels = {}
        G.remove_edges_from(nx.selfloop_edges(G))
        all_nodes = G.nodes
        for key in all_nodes:
            labels[key] = key

        nx.draw_networkx_labels(G, pos, labels, font_size=8)
        nx.draw(G, pos, font_size=1)
        
        plt.title("Correlation Graph")
        plt.show()

In [ ]:
graph_generator(dt)